<a href="https://colab.research.google.com/github/ranjan2829/Quant/blob/main/Quant_%26_Algorithmic_trading_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Unsupervised Learning on s&p 500 stocks


In [4]:
pip install pandas_ta


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=ddde82eb467149671d6cdace5034e6f09a7e45e269a29a7b0ebe9e8d63570fbe
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [6]:
pip install statsmodels


In [8]:
import pandas as pd
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
import matplotlib.pyplot as plt
import pandas_datareader.data as web
from statsmodels.regression.rolling import RollingOLS

warnings.filterwarnings('ignore')

In [11]:
sp=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

In [12]:
sp


,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
501,ZION,Zions Bancorporation,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [13]:
sp['Symbol']=sp['Symbol'].str.replace('.','-')

In [14]:
symbollist=sp['Symbol'].unique().tolist()

In [15]:
endd='2023-12-24'
startd='2023-01-2'
data=yf.download(tickers=symbollist,start=startd,end=endd)

[*********************100%%**********************]  503 of 503 completed


In [16]:
data



Adj Close                                                         \
                     A    AAL        AAPL        ABBV        ABNB         ABT   
Date                                                                            
2023-01-03  149.209610  12.74  124.374809  156.074890   84.900002  107.430855   
2023-01-04  150.830612  13.59  125.657639  157.334015   88.720001  109.028885   
2023-01-05  151.268173  13.99  124.325089  157.141785   87.709999  108.626930   
2023-01-06  146.852737  14.18  128.899506  160.082977   88.519997  110.126923   
2023-01-09  146.653854  14.61  129.426544  155.382843   89.239998  109.950455   
...                ...    ...         ...         ...         ...         ...   
2023-12-18  137.529999  14.24  195.889999  153.419998  147.500000  108.059998   
2023-12-19  139.789993  14.30  196.940002  153.600006  146.910004  108.419998   
2023-12-20  138.179993  13.98  194.830002  151.690002  141.070007  107.400002   
2023-12-21  138.940002  14.35  194.679993  152.589996  142.089996  108.480003   
2023-12-22  139.570007  14.31  193.600006  154.940002  140.800003  109.029999   

                                                           ...   Volume  \
                 ACGL         ACN        ADBE         ADI  ...     WYNN   
Date                                                       ...            
2023-01-03  62.459999  266.033325  336.920013  159.414185  ...  3318600   
2023-01-04  62.770000  265.127686  341.410004  162.809555  ...  5743600   
2023-01-05  62.939999  258.867188  328.440002  156.705780  ...  2983000   
2023-01-06  63.900002  264.999725  332.750000  162.426834  ...  2901900   
2023-01-09  62.669998  269.468750  341.980011  163.977325  ...  2751000   
...               ...         ...         ...         ...  ...      ...   
2023-12-18  74.650002  341.850006  599.130005  196.589996  ...  1487600   
2023-12-19  74.830002  341.510010  604.640015  197.830002  ...  2264100   
2023-12-20  73.610001  339.500000  596.059998  192.190002  ...  2039600   
2023-12-21  74.160004  348.750000  600.140015  196.279999  ...  1891800   
2023-12-22  73.839996  354.450012  598.750000  197.100006  ...  1170500   

                                                                           \
                XEL       XOM     XRAY      XYL      YUM      ZBH    ZBRA   
Date                                                                        
2023-01-03  2684500  15146200  1653800   812400  2307100   921100  295400   
2023-01-04  2265900  18058400  3697800   905900  1139400  1389400  311600   
2023-01-05  2104500  15946600  4455700  1295700  1078200  2238300  368200   
2023-01-06  2976200  16348100  2058700   636200  1559400  2348900  552700   
2023-01-09  2707800  17964600  1468900  1224700  1211200  1423600  535400   
...             ...       ...      ...      ...      ...      ...     ...   
2023-12-18  6632200  21610400  2226900  1071200  1883900  2075000  485000   
2023-12-19  6398800  18904700  3449100  1547000  1704200  1508900  467000   
2023-12-20  4264500  20988900  1652400  1442900  1321400  3205400  332800   
2023-12-21  2807700  19250900  2604200  1037000  1211600  1971100  282500   
2023-12-22  2429300  12921800  1261200   829300   991500  1337800  225900   

                              
               ZION      ZTS  
Date                          
2023-01-03  1175800  1840600  
2023-01-04  1689000  1796100  
2023-01-05   867800  1560900  
2023-01-06  1363700  1761200  
2023-01-09   902700  1700500  
...             ...      ...  
2023-12-18  2066600  1543100  
2023-12-19  1876400  1600300  
2023-12-20  2062200  1833200  
2023-12-21  1728300  2118600  
2023-12-22  1534500  1548400  

[246 rows x 3018 columns]